## Topic Modelling

We have downloaded some speeches by B. Obama and M. Romney made during US Presidential election 2012. The speeches are uploaded on GitHUB and we would try to find some imortant topics each of their speeches consisted of. We will do Topic modelling using Gensim and visualize these topics using a womderful package called pyLDAvis.

**NOTE:** If pyLDAvis visualization diagrams are NOT visible in this notebook, refer to this jupyter nbviewer version 
https://nbviewer.jupyter.org/github/Rajesh-Nair/Text-modelling-on-speeches/blob/master/Topic%20Modelling%20using%20gensim%20%28LSI%20and%20LDA%29.ipynb


In [30]:
# Find folders and text under a given path
import os

# For dataframe/tables
import pandas as pd

# NLTK package
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Gensim
from gensim import corpora
from gensim import models

# Visualization
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [31]:
# Text document folder
folder = 'speeches'

# Stop words set
ignorewords = set(stopwords.words("english"))

# Lemmatizer
lemmatizer = WordNetLemmatizer().lemmatize

#define dictionary  and corpus
dictionary = corpora.Dictionary()
corpus = list()

# Clean text
# 1. Word Tokenize text
# 2. Ignore stopwords and non alpha characters, words less than 4 in length
# 3. conver to lower case
# 4. Lemmatize the words
# 5. Update/Add to Dictionary
# 6. Create Bag of Words
def clean_corpus(text) :
    textlist = list()
    textlist.append([lemmatizer(word.lower()) for word in word_tokenize(text) \
                       if word not in ignorewords and word.isalpha() and len(word) > 4 ])
    dictionary.add_documents(textlist)
    for text in textlist :
        corpus.append(dictionary.doc2bow(text, allow_update=True))    
    return(None)



In [32]:
# Read, clean and convert the speeches in to Tf-idf
corpus_tfidf = dict()
for fld in  os.listdir(folder) :
    corpus = [] # Re-Initialize    
    for file in os.listdir(folder + '\\' + fld) :
        txtfile = folder + '\\' + fld + '\\' + file
        with open(txtfile, 'r',errors='ignore') as fobj :        
            clean_corpus(fobj.read())       
    # Tf-idf model
    Tfidf = models.TfidfModel(corpus, smartirs='ntc')
    corpus_tfidf[fld] = Tfidf[corpus]    

# Store the dictionary
corpus_dict = dictionary

In [33]:
# LSI model
lsi = dict()
num_topics = 5 
num_words = 15

for fld in corpus_tfidf.keys() :
    lsi[fld] = models.LsiModel(corpus_tfidf[fld], id2word=corpus_dict, num_topics=num_topics)


In [34]:
# Display the topics and words associated
for fld in corpus_tfidf.keys() :    
    print('*'*80)
    print(fld.upper())
    print(pd.DataFrame({"Topic "+ str(i+1):[topic[0] for topic in lsi[fld].show_topic(i,num_words)] \
                                    for i in range(num_topics)}, index = [i+1 for i in range(num_words)] ))

********************************************************************************
ROMNEY
    Topic 1     Topic 2       Topic 3     Topic 4       Topic 5
1   liberal       found         reply       peace      applause
2     found   searching        cancel      regime       dreamer
3    crisis   requested         leave        ally         plant
4   program     perhaps        falcon     dreamer        saving
5    school     nothing         berta      saving      remember
6    budget       could       library      weapon        little
7     going     liberal        valley    applause  unidentified
8    little       going      provided     veteran      audience
9    dollar      crisis         tweet       plant         going
10   nation      budget          view     iranian     attacking
11  percent       reply  presidential       build    imaginable
12    build    strategy      applause        wage         meant
13  company      little        ronald  capability    unfairness
14   always  str

In [35]:
# LDA model
lda = dict()
num_topics = 5 
num_words = 15
num_of_passes = 20

for fld in corpus_tfidf.keys() :
    lda[fld] = models.LdaModel(corpus_tfidf[fld], id2word=corpus_dict, num_topics=num_topics, passes = num_of_passes)


In [36]:
# Display the topics and words associated
for fld in corpus_tfidf.keys() :  
    print('*'*80)
    print(fld.upper())
    print(pd.DataFrame({"Topic "+ str(i+1):[topic[0] for topic in lda[fld].show_topic(i,num_words)] \
                                    for i in range(num_topics)}, index = [i+1 for i in range(num_words)] ))

********************************************************************************
ROMNEY
         Topic 1    Topic 2       Topic 3       Topic 4   Topic 5
1       applause      found         reply     christian   comment
2        liberal  searching        cancel         jerry  remember
3          going  requested         leave  unidentified    closed
4        teacher    perhaps         tweet        christ    crisis
5         school      berta          view         chuck     peace
6   unidentified     falcon       prevail       logical   liberal
7        student    nothing      indicate        truett    weapon
8         choice      could       ethical       worldly  strategy
9          build      kerry        regime  christianity   mission
10       percent      derek       citadel      cheerful    regime
11        little    vietnam      granholm     spiritual     three
12       library     desert     criticism       trivial      ally
13        always    winning       israeli       falwel

In [37]:
# Visualize - LDA using pyLDAvis
vis = dict()
for fld in corpus_tfidf.keys() :
    vis[fld] = pyLDAvis.gensim.prepare(lda[fld],corpus=corpus_tfidf[fld],dictionary=corpus_dict)

C:\Users\acer\Anaconda3\envs\py35\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


In [38]:
# Print topics in Romney's speeches
print(list(corpus_tfidf.keys())[0])
vis[list(corpus_tfidf.keys())[0]]

romney


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      48.268745        1       1  0.007015  0.004639
4      47.058769        1       2  0.004671 -0.005894
1       1.930462        1       3 -0.004549  0.000536
2       1.486977        1       4 -0.003670  0.000403
3       1.255046        1       5 -0.003468  0.000316, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
6985  Default  0.000000         reply  0.000000  30.0000  30.0000
4660  Default  0.000000         found  0.000000  29.0000  29.0000
85    Default  0.000000        cancel  0.000000  28.0000  28.0000
4190  Default  0.000000     requested  0.000000  27.0000  27.0000
482   Default  0.000000     searching  0.000000  26.0000  26.0000
6932  Default  0.000000       perhaps  0.000000  25.0000  25.0000
3141  Default  0.000000       nothing  0.000000  24.0000  24.0000
1256  Default  0.000000         leave  0.000000  23.0000  23.0000
5217  Default  0.000000         berta  0.000000  22.0000  22.0000
2593  Default  0.000000        falcon  0.000000  21.0000  21.0000
5762  Default  0.000000         could  0.000000  20.0000  20.0000
6042  Default  0.000000      applause  0.000000  19.0000  19.0000
996   Default  0.000000         kerry  0.000000  18.0000  18.0000
7227  Default  0.000000  unidentified  0.000000  17.0000  17.0000
2881  Default  0.000000        regime  0.000000  16.0000  16.0000
8859  Default  0.000000       library  0.000000  15.0000  15.0000
5148  Default  0.000000       mission  0.000000  14.0000  14.0000
6758  Default  0.000000        weapon  0.000000  13.0000  13.0000
739   Default  0.000000      remember  0.000000  12.0000  12.0000
3005  Default  0.000000      audience  0.000000  11.0000  11.0000
7371  Default  0.000000         plant  0.000000  10.0000  10.0000
339   Default  0.000000         peace  0.000000   9.0000   9.0000
6116  Default  0.000000  presidential  0.000000   8.0000   8.0000
6556  Default  0.000000       student  0.000000   7.0000   7.0000
4035  Default  0.000000         torch  0.000000   6.0000   6.0000
5764  Default  0.000000    unfairness  0.000000   5.0000   5.0000
5815  Default  0.000000      mortgage  0.000000   4.0000   4.0000
8314  Default  0.000000       iranian  0.000000   3.0000   3.0000
663   Default  0.000000       comment  0.000000   2.0000   2.0000
3902  Default  0.000000          adam  0.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
3230   Topic5  0.001260     worldview  0.091064   0.0977  -9.1855
7168   Topic5  0.001260       censure  0.091064   0.0977  -9.1855
7835   Topic5  0.001260       recedes  0.091064   0.0977  -9.1855
7910   Topic5  0.001260      remarked  0.091064   0.0977  -9.1855
6931   Topic5  0.001260    comforting  0.091064   0.0977  -9.1855
1071   Topic5  0.001260         cathy  0.091064   0.0977  -9.1855
5027   Topic5  0.001260        wesley  0.091064   0.0977  -9.1855
4123   Topic5  0.001260    resurgence  0.091064   0.0977  -9.1855
4321   Topic5  0.001275         drama  0.105133  -0.0340  -9.1735
6099   Topic5  0.001281      observed  0.115222  -0.1208  -9.1687
7227   Topic5  0.001339  unidentified  0.227709  -0.7582  -9.1248
6123   Topic5  0.001271         jesus  0.104856  -0.0347  -9.1768
4878   Topic5  0.001275     completed  0.110404  -0.0832  -9.1737
6309   Topic5  0.001277        talent  0.117473  -0.1439  -9.1724
3638   Topic5  0.001276      separate  0.117804  -0.1475  -9.1732
5176   Topic5  0.001275      devotion  0.118560  -0.1543  -9.1736
8349   Topic5  0.001264          open  0.097123   0.0362  -9.1826
3357   Topic5  0.001264        recall  0.097123   0.0362  -9.1826
208    Topic5  0.001264        luther  0.097123   0.0362  -9.1826
3715   Topic5  0.001264        martin  0.097123   0.0362  -9.1826
1168   Topic5  0.001264      theology  0.097123   0.0362  -9.1826
4258   Topic5  

In [39]:
# Print topics in Obama's speeches
print(list(corpus_tfidf.keys())[1])
vis[list(corpus_tfidf.keys())[1]]

obama


PreparedData(topic_coordinates=            Freq  cluster  topics         x             y
topic                                                    
0      94.465958        1       1  0.023943  1.294720e-07
3       1.601733        1       2 -0.006266  1.156224e-05
1       1.310905        1       3 -0.005895 -8.513481e-05
4       1.310707        1       4 -0.005879  4.321792e-05
2       1.310687        1       5 -0.005903  3.022518e-05, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
3526  Default  0.000000      pandemic  0.000000  30.0000  30.0000
5022  Default  0.000000      employer  0.000000  29.0000  29.0000
6913  Default  0.000000       teacher  0.000000  28.0000  28.0000
7513  Default  0.000000        credit  0.000000  27.0000  27.0000
8351  Default  0.000000        senate  0.000000  26.0000  26.0000
1865  Default  0.000000      industry  0.000000  25.0000  25.0000
5666  Default  0.000000        system  0.000000  24.0000  24.0000
2714  Default  0.000000  undocumented  0.000000  23.0000  23.0000
1970  Default  0.000000          fuel  0.000000  22.0000  22.0000
4626  Default  0.000000       disease  0.000000  21.0000  21.0000
1459  Default  0.000000         voter  0.000000  20.0000  20.0000
6758  Default  0.000000        weapon  0.000000  19.0000  19.0000
8824  Default  0.000000       company  0.000000  18.0000  18.0000
6963  Default  0.000000    transcript  0.000000  17.0000  17.0000
1714  Default  0.000000         going  0.000000  16.0000  16.0000
5182  Default  0.000000        family  0.000000  15.0000  15.0000
9467  Default  0.000000        record  0.000000  14.0000  14.0000
7791  Default  0.000000      standard  0.000000  13.0000  13.0000
3475  Default  0.000000         think  0.000000  12.0000  12.0000
3161  Default  0.000000       million  0.000000  11.0000  11.0000
866   Default  0.000000    washington  0.000000  10.0000  10.0000
5623  Default  0.000000         force  0.000000   9.0000   9.0000
4226  Default  0.000000       benefit  0.000000   8.0000   8.0000
6556  Default  0.000000       student  0.000000   7.0000   7.0000
2724  Default  0.000000       believe  0.000000   6.0000   6.0000
1988  Default  0.000000          cost  0.000000   5.0000   5.0000
402   Default  0.000000        threat  0.000000   4.0000   4.0000
5233  Default  0.000000        chance  0.000000   3.0000   3.0000
6404  Default  0.000000        mccain  0.000000   2.0000   2.0000
2418  Default  0.000000         dream  0.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
2446   Topic5  0.002031   unjustified  0.136142   0.1297  -9.1464
1488   Topic5  0.002031     unbending  0.136143   0.1297  -9.1464
524    Topic5  0.002111         sudan  0.189916  -0.1646  -9.1077
8166   Topic5  0.002025       pledged  0.136667   0.1225  -9.1497
527    Topic5  0.002025      insanity  0.136667   0.1225  -9.1497
1250   Topic5  0.002025       germane  0.136667   0.1225  -9.1497
4552   Topic5  0.002251         cairo  0.379339  -0.7923  -9.0436
5860   Topic5  0.002098         rebel  0.193720  -0.1910  -9.1143
3018   Topic5  0.002065  construction  0.169499  -0.0729  -9.1298
3572   Topic5  0.002202        coburn  0.336692  -0.6953  -9.0659
1039   Topic5  0.002045     adherence  0.161740  -0.0362  -9.1399
2785   Topic5  0.002032      breeding  0.151990   0.0200  -9.1460
4255   Topic5  0.002046      khartoum  0.166084  -0.0619  -9.1391
2456   Topic5  0.002100       amnesty  0.222355  -0.3278  -9.1133
4925   Topic5  0.001977        untied  0.117905   0.2465  -9.1734
7586   Topic5  0.001977      hardware  0.117905   0.2465  -9.1734
5388   Topic5  0.002145     displaced  0.316417  -0.6595  -9.0922
6198   Topic5  0.002010         cubic  0.142334   0.0747  -9.1568
4053   Topic5  0.002057       refugee  0.199278  -0.2390  -9.1341
4877   Topic5  0.002111          case  0.352179  -0.7824  -9.1080
2714   Topic5  0.002119  undocumented  0.511830  -1.1